In [23]:
from uuid import uuid4
from datetime import datetime
from typing import Annotated, Any

import pytz
from dateutil.parser import parse

from pydantic import BaseModel, ConfigDict, EmailStr, Field, PastDate, AfterValidator, BeforeValidator, FieldSerializationInfo, PlainSerializer
from pydantic.alias_generators import to_camel

In [16]:
class CustomBaseModel(BaseModel):
    model_config = ConfigDict(
        extra="ignore",
        alias_generator=to_camel,
        populate_by_name=True,
    )

In [17]:
SortedStringList = Annotated[list[str], AfterValidator(lambda value: sorted(value, key=str.casefold))]

class ContactInfo(CustomBaseModel):
    email: EmailStr | None = None

class PlaceInfo(CustomBaseModel):
    city: str
    country: str
    
class PlaceDateInfo(CustomBaseModel):
    date_: PastDate = Field(alias="date")
    place: PlaceInfo
    
class PersonalInfo(CustomBaseModel):
    nationality: str
    born: PlaceDateInfo

class Person(CustomBaseModel):
    first_name: str
    last_name: str
    contact_info: ContactInfo
    personal_info: PersonalInfo
    notable_students: SortedStringList = []

In [18]:
json_data = """
{
    "firstName": "David",
    "lastName": "Hilbert",
    "contactInfo": {
        "email": "d.hilbert@spectral-theory.com",
        "homePhone": {
            "countryCode": 49,
            "areaCode": 551,
            "localPhoneNumber": 123456789
        }
    },
    "personalInfo": {
        "nationality": "German",
        "born": {
            "date": "1862-01-23",
            "place": {
                "city": "Konigsberg",
                "country": "Prussia"
            }
        },
        "died": {
            "date": "1943-02-14",
            "place": {
                "city": "Gottingen",
                "country": "Germany"
            }
        }
    },
    "awards": ["Lobachevsky Prize", "Bolyai Prize", "ForMemRS"],
    "notableStudents": ["von Neumann", "Weyl", "Courant", "Zermelo"]
}
"""

In [19]:
p = Person.model_validate_json(json_data)

Mario
first_name='David' last_name='Hilbert' contact_info=ContactInfo(email='d.hilbert@spectral-theory.com') personal_info=PersonalInfo(nationality='German', born=PlaceDateInfo(date_=datetime.date(1862, 1, 23), place=PlaceInfo(city='Konigsberg', country='Prussia'))) notable_students=['Courant', 'von Neumann', 'Weyl', 'Zermelo']


In [20]:
class CustomBaseModel(BaseModel):
    model_config = ConfigDict(
        extra="ignore",
        alias_generator=to_camel,
        populate_by_name=True,
    )

In [21]:

def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt
    
def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value

def dt_serializer(dt, info: FieldSerializationInfo) -> datetime | str:
    if info.mode_is_json():
        return dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    return dt
    
DateTimeUTC = Annotated[
    datetime, 
    BeforeValidator(parse_datetime), 
    AfterValidator(make_utc), 
    PlainSerializer(dt_serializer, when_used="unless-none")
]

In [24]:
class RequestInfo(CustomBaseModel):
    query_id: uuid4 = Field(default_factory=uuid4)
    execution_dt: DateTimeUTC = Field(default_factory=lambda: datetime.now(pytz.utc))
    elapsed_time_secs: float = 0

In [25]:
class ResponseBaseModel(CustomBaseModel):
    request_info: RequestInfo

In [26]:
class Users(ResponseBaseModel):
    users: list[str] = []

In [27]:
users = Users(request_info=RequestInfo(elapsed_time_secs=10), users=["Mario", "Luigi"])

In [28]:
users

Users(request_info=RequestInfo(query_id=UUID('a37b3c16-432b-4050-9687-f566bf005bc6'), execution_dt=datetime.datetime(2024, 8, 29, 13, 15, 52, 304637, tzinfo=<UTC>), elapsed_time_secs=10.0), users=['Mario', 'Luigi'])